In [1]:
import sys

sys.version

'3.6.0 |Anaconda 4.3.1 (64-bit)| (default, Dec 23 2016, 11:57:41) [MSC v.1900 64 bit (AMD64)]'

In [2]:
import os, numpy as np, operator, math, json
from pprint import pprint
import tflearn, sys, tensorflow as tf

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


### Environment Variables

In [3]:
ex_num = 13
filename = 'seq10_batch10_01'
description = "IFC 파일 한번씩 훈련시키기 2,3,4,5,6,7,8,9,10 로 훈련시키고 1로 테스트하기"
train_data_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10]
test_ifc_ids = [1]
seqlen = 10
lstmhid=150
keeprate = 0.80
train = []
true = []
X = []
Y = []
gross_len = 100
ft_num = 8
tf.reset_default_graph()
batch_size = 10
n_epoch = 100
datas_path = 'D:\\projects\\capstone\\notebook\\experiments\\datas'

In [4]:
with open(datas_path+"\\train_datas.json") as data_file:
    train_datas = json.load(data_file)
    
with open(datas_path+"\\test_datas.json") as data_file:
    test_datas = json.load(data_file)

In [4]:
print(len(train_datas))
print(len(test_datas))

9
1


In [5]:
# input : list of parsing data ( ifc_num * wall_num * wall_feature(8) )
# output :
#       X : list, (ifc_num * ndarray)
#       Y : list, (ifc_num * ndarray)
def gen_dataset(dataset):
    X = []
    Y = []
    dataset_fix = []
    for j in range(len(dataset)):
        dataset_one = []
        dataset_pre = []
        dataset_pre = dataset[j].copy()
        
        switch = 0
        pre_size = len(dataset[j])

        while len(dataset[j]) < gross_len:
            count = 0
            if switch == 0 :
                for i in range(pre_size-1):
                    dataset[j].append(dataset_pre[pre_size-2-i])
                switch = 1
                count = count + pre_size - 1
            else:
                for i in range(pre_size-1):
                    dataset[j].append(dataset_pre[i + 1])
                switch = 0
                count = count + pre_size - 1
        
        print("len(dataset[j]) :", len(dataset[j]))
        print("grosslen :", gross_len)
        
        for i in range(gross_len):
            dataset_one.append(dataset[j][i])
        
        dataset_fix.append(dataset_one)

        for i in range(0, len(dataset_fix[j])):    
            if i % 20 == 0 :
                print("-")
            pprint(dataset_fix[j][i][0])
            if i % 20 == 0 :
                print("-")
                
    for j in range(0, len(dataset_fix)):
        train = []
        true = []
        for i in range(0, len(dataset_fix[j])-seqlen, 1):
            train.append([wall for wall in dataset_fix[j][i:i+seqlen]])
            true.append(dataset_fix[j][i+seqlen])
        X.append(np.reshape(train, (len(train), seqlen, 8)))
        Y.append(np.reshape(true, (len(true), 8)))
        
    return X, Y

In [6]:
# ifc별 X, Y 데이터 리스트
X, Y = gen_dataset(train_datas)

len(dataset[j]) : 121
grosslen : 100
-
0.0
-
0.0
4.074999999999999
0.07499999999999929
0.15000000000000036
11.100000000000001
2.4749999999999
2.5499999999998995
4.175000000000001
4.15
2.399999999999901
11.175
5.275
5.875
8.375
7.800000000000001
7.949999999999999
8.45
7.800000000000001
5.35
-
0.15000000000000036
-
7.875
3.25
3.1750000000000007
1.6749999999998995
3.1750000000000007
3.25
7.875
0.15000000000000036
5.35
7.800000000000001
8.45
7.949999999999999
7.800000000000001
8.375
5.875
5.275
11.175
2.399999999999901
4.15
-
4.175000000000001
-
2.5499999999998995
2.4749999999999
11.100000000000001
0.15000000000000036
0.07499999999999929
4.074999999999999
0.0
0.0
0.0
4.074999999999999
0.07499999999999929
0.15000000000000036
11.100000000000001
2.4749999999999
2.5499999999998995
4.175000000000001
4.15
2.399999999999901
11.175
-
5.275
-
5.875
8.375
7.800000000000001
7.949999999999999
8.45
7.800000000000001
5.35
0.15000000000000036
7.875
3.25
3.1750000000000007
1.6749999999998995
3.17500000000

In [7]:
if str(ex_num)+"_"+filename in os.listdir(os.getcwd()):
    pass
else:
    os.mkdir(os.getcwd()+"/"+str(ex_num)+"_"+filename)

In [8]:
net = tflearn.input_data(shape=(None, seqlen, ft_num))
net = tflearn.lstm(net, lstmhid)
net = tflearn.fully_connected(net, ft_num, activation='relu')
net = tflearn.regression(net, optimizer='adam',
learning_rate=0.002, loss ='mean_square')
model = tflearn.DNN(net, checkpoint_path = os.getcwd()+"\\"+str(ex_num)+"_"+filename+'\\model_01.tfl.ckpt')
model.save(os.getcwd()+"\\"+str(ex_num)+"_"+filename)

INFO:tensorflow:D:\projects\capstone\notebook\experiments\13_seq10_batch10_01 is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
# model.load(os.getcwd()+"\\"+str(ex_num)+"_"+filename)
for i in range(len(X)):
    print(len(X))
    print(X[0].shape)
    print(Y[0].shape)
    model.fit(X[i], Y[i], snapshot_epoch=True, snapshot_step=5000, n_epoch=100, batch_size=batch_size)
    model.save(os.getcwd()+"\\"+str(ex_num)+"_"+filename)
        

Training Step: 8099  | total loss: 0.52017 | time: 0.064s
| Adam | epoch: 900 | loss: 0.52017 -- iter: 80/90
Training Step: 8100  | total loss: 0.49245 | time: 0.072s
| Adam | epoch: 900 | loss: 0.49245 -- iter: 90/90
--
INFO:tensorflow:D:\projects\capstone\notebook\experiments\13_seq10_batch10_01\model_01.tfl.ckpt-8100 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:D:\projects\capstone\notebook\experiments\13_seq10_batch10_01\model_01.tfl.ckpt-8100 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:D:\projects\capstone\notebook\experiments\13_seq10_batch10_01 is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
test_X, test_Y = gen_dataset(test_datas)

len(dataset[j]) : 109
grosslen : 100
-
0.6749999999999972
-
0.07499999999999574
2.0
8.174999999999997
5.674999999999997
2.0749999999999957
0.6749999999999972
8.100000000000001
8.100000000000001
14.0
6.574999999999996
6.5
1.4750000000000014
4.149999999999999
4.074999999999996
6.100000000000001
5.600000000000001
0.14999999999999858
14.074999999999996
6.174999999999997
-
0.75
-
8.174999999999997
5.274999999999999
14.475000000000001
14.074999999999996
12.375
14.399999999999999
12.375
14.399999999999999
12.375
14.074999999999996
14.475000000000001
5.274999999999999
8.174999999999997
0.75
6.174999999999997
14.074999999999996
0.14999999999999858
5.600000000000001
6.100000000000001
-
4.074999999999996
-
4.149999999999999
1.4750000000000014
6.5
6.574999999999996
14.0
8.100000000000001
8.100000000000001
0.6749999999999972
2.0749999999999957
5.674999999999997
8.174999999999997
2.0
0.07499999999999574
0.6749999999999972
0.07499999999999574
2.0
8.174999999999997
5.674999999999997
2.0749999999999957

In [6]:
net = tflearn.input_data(shape=(None, seqlen, ft_num))
net = tflearn.lstm(net, lstmhid)
net = tflearn.fully_connected(net, ft_num, activation='relu')
net = tflearn.regression(net, optimizer='adam',
learning_rate=0.002, loss ='mean_square')
model = tflearn.DNN(net, checkpoint_path = os.getcwd()+"\\"+str(ex_num)+"_"+filename+'\\model_01.tfl.ckpt')
model.load(".\\"+str(ex_num)+"_"+filename+'\\model_01.tfl.ckpt-8100')

INFO:tensorflow:Restoring parameters from D:\projects\capstone\notebook\experiments\13_seq10_batch10_01\model_01.tfl.ckpt-8100


NotFoundError: Key LSTM_1/LSTM_1/BasicLSTMCell/Linear/Matrix not found in checkpoint
	 [[Node: save_5/RestoreV2_30 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_5/Const_0_0, save_5/RestoreV2_30/tensor_names, save_5/RestoreV2_30/shape_and_slices)]]

Caused by op 'save_5/RestoreV2_30', defined at:
  File "D:\envir\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\envir\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\envir\anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\envir\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\envir\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "D:\envir\anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    sup

In [12]:
test_result = []
test_times = 3
generate_num = 10

for _ in range(test_times):
    p = test_X[0][np.random.randint(0,test_X[0].shape[0]-1)]
    print("Seed:")
    pprint(p.tolist())
    case = []

    for wall in p.tolist():
        case.append([[wall[0], wall[1], wall[2]], [wall[3], wall[4], wall[5]]])
        
    print("\n")
    
    for _ in range(generate_num):
        
        pprint(model.predict(np.reshape(p, (1, seqlen, 8))).tolist())
        
        for wall in model.predict(np.reshape(p, (1, seqlen, 8))).tolist():
            case.append([[wall[0], wall[1], wall[2]], [wall[3], wall[4], wall[5]]])
        
        p = np.concatenate((p, model.predict(np.reshape(p, (1, seqlen, 8)))))
        p = p[1:len(p)]
    
    test_result.append(case)
    print("\n============================\n")

Seed:
[[5.600000000000001,
  4.875,
  0.0,
  2.1499999999999986,
  4.875,
  0.0,
  3.450000000000003,
  3.141592653589793],
 [0.14999999999999858,
  10.675000000000004,
  0.0,
  5.199999999999996,
  10.675000000000004,
  0.0,
  5.049999999999997,
  0.0],
 [14.074999999999996,
  0.15000000000000568,
  0.0,
  14.074999999999996,
  4.800000000000004,
  0.0,
  4.649999999999999,
  1.5707963267948966],
 [6.174999999999997,
  13.900000000000002,
  0.0,
  6.174999999999997,
  12.000000000000004,
  0.0,
  1.8999999999999986,
  1.5707963267948966],
 [0.75,
  0.07500000000000284,
  0.0,
  14.149999999999999,
  0.07500000000000284,
  0.0,
  13.399999999999999,
  0.0],
 [8.174999999999997,
  13.900000000000002,
  0.0,
  8.174999999999997,
  10.59999999999998,
  0.0,
  3.300000000000022,
  1.5707963267948966],
 [5.274999999999999,
  12.000000000000004,
  0.0,
  5.274999999999999,
  10.250000000000007,
  0.0,
  1.7499999999999964,
  1.5707963267948966],
 [14.475000000000001,
  4.950000000000003,
  0

RuntimeError: Attempted to use a closed Session.

In [12]:
for i in range(test_times):
    print(i, "'s test : ")
    pprint(test_result[i])

0 's test : 
[[[8.174999999999997, 13.900000000000002, 0.0],
  [8.174999999999997, 10.59999999999998, 0.0]],
 [[5.274999999999999, 12.000000000000004, 0.0],
  [5.274999999999999, 10.250000000000007, 0.0]],
 [[14.475000000000001, 4.950000000000003, 0.0],
  [14.475000000000001, 10.75, 0.0]],
 [[3.190307140350342, 5.504159450531006, 0.0],
  [8.284193992614746, 7.126020908355713, 0.0]],
 [[1.3605165481567383, 0.7025571465492249, 0.0],
  [5.940084457397461, 0.7249197959899902, 0.0]],
 [[2.2713053226470947, 2.583212375640869, 0.0], [0.0, 9.474478721618652, 0.0]],
 [[1.508089542388916, 7.844461441040039, 0.0],
  [0.9124026298522949, 8.768021583557129, 0.0]],
 [[0.0, 9.972497940063477, 0.0], [1.7403579950332642, 10.606904983520508, 0.0]],
 [[1.873395323753357, 9.951040267944336, 0.0],
  [3.8954577445983887, 6.354604244232178, 0.0]],
 [[2.5889551639556885, 11.595582962036133, 0.0],
  [5.635130405426025, 11.655776977539062, 0.0]],
 [[3.0442488193511963, 8.892464637756348, 0.0],
  [5.154442787170